**BINOMIAL TEST**

In [8]:
from scipy.stats import binomtest

# Perform a BINOMIAL TEST to assess whether the observed number of correct trials
# significantly deviates from the expected chance level.

# The observed data (correct trials out of total) comes from Table 2 in the paper.
# The chance levels for each task are defined in the Methods section.


observed_successes = 4  # Number of correct responses (from Table 2)
total_trials = 30        # Total number of trials (from Table 2)
chance_level = 0.25      # chance-level of the task (in Methods)


result = binomtest(observed_successes, total_trials, chance_level, alternative='two-sided')
p_value = result.pvalue


print(p_value)
alpha = 0.05

if p_value < alpha:
    print("Performance is significantly different from chance (p < 0.05).")
else:
    print("Performance is NOT significantly different from chance.")

0.20359772225731598
Performance is NOT significantly different from chance.


**PERMUTATION TEST**

In [9]:
import numpy as np

# set a random seed for reproducibility
np.random.seed(42)

# For tasks without a well-defined chance level (e.g., WISC-R Block Design and Hand Gestures tasks),
# we use permutation testing to assess whether performance with the left hand significantly differs from the right hand.

# Input the data for each hand as a binary array, where 1 represents a correct response
# and 0 represents an incorrect response. The data is from Table 2 in the paper.
# Example: performance of 3/4 is represented as [1,1,1,0].

right_hand = [1,1,1,0]
left_hand = [0,0,0]

# Get prop correct per hand and then their differences
prop_right = np.mean(right_hand)
prop_left = np.mean(left_hand)
observed_diff = abs(prop_right - prop_left)

# Combine all data
combined_data = np.array(right_hand + left_hand)
n_right = len(right_hand)
n_left = len(left_hand)

# Permutation testing
n_iterations = 10000
perm_stats = []
for i in range(n_iterations):
    np.random.shuffle(combined_data)
    perm_right = combined_data[:n_right] # assign the first part (length = right hand array) to right hand
    perm_left = combined_data[n_right:] # assign the remaining part to left hand
    perm_stat = abs(np.mean(perm_right) - np.mean(perm_left))
    perm_stats.append(perm_stat)

# Calculate p-value
perm_stats = np.array(perm_stats)
p_value = np.sum(perm_stats >= observed_diff) / n_iterations

print(f"Observed Difference: {observed_diff}")
print(f"p-value: {p_value}")

Observed Difference: 0.75
p-value: 0.1489


**MONTE CARLO SIMULATION**

In [10]:
import numpy as np

# Random seed for reproducibility
np.random.seed(42)

# Observed data
observed_correct = 4
total_trials = 30
chance_prob= 0.5

num_permutations = 10000

# observed accuracy
observed_accuracy = observed_correct / total_trials

# Generate accuracy under the null hypothesis
def generate_null_distribution(total_trials, num_permutations, prob=chance_prob):
    null_distribution = []
    for i in range(num_permutations):
        simulated_data = np.random.binomial(n=1, p=prob, size=total_trials)
        null_accuracy = np.sum(simulated_data) / total_trials
        null_distribution.append(null_accuracy)
    return null_distribution

# Generate null distribution
null_distribution = generate_null_distribution(total_trials, num_permutations, prob=chance_prob)

# p-value (two-sided)
extreme_count = np.sum(np.abs(np.array(null_distribution) - chance_prob) >= np.abs(observed_accuracy - chance_prob))
p_value = extreme_count / num_permutations


# Results
print(p_value)
alpha = 0.05
if p_value < alpha:
    print("different from chance (p < 0.05).")
else:
    print("NOT different from chance.")

0.0
different from chance (p < 0.05).
